# Stance Detection Using MBERT + CNN

`Multi-lingual Bidirectional Encoder Representations from Transformers` `Convolutional Neural Networks`  
`AraStance Dataset`  
`Stance Detection` `Arabic Language` `Transformer Architecture`


---

In this notebook, we rely on the features extracted from the last four layers of the multi-lingual version of BERT to classify the stances of the articles in the AraStance dataset. The dataset was introduced in the paper:
```
AraStance: A Multi-Country and Multi-Domain Dataset of Arabic Stance Detection for Fact Checking.
```
The model was introduced in the paper:
```
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
```

## Packages

In [ ]:
!pip install transformers

In [ ]:
import torch
import numpy as np
from utils import *
from tqdm.auto import tqdm
from torch.optim import AdamW
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

## Raw data

- Download the raw data:

In [ ]:
!wget https://github.com/Tariq60/arastance/archive/refs/heads/main.zip
!unzip /content/main.zip

- Read the data:

In [ ]:
raw_train = AraStanceData("/content/arastance-main/data/train.jsonl")
raw_val = AraStanceData("/content/arastance-main/data/dev.jsonl")
raw_test = AraStanceData("/content/arastance-main/data/test.jsonl")

print(f'# training instances: {len(raw_train.stances)}')
print(f'# validation instances: {len(raw_val.stances)}')
print(f'# testing instances: {len(raw_test.stances)}')

# training instances: 2848
# validation instances: 569
# testing instances: 646


- Print an instance from the data

In [ ]:
instance_no = 40
print(f"Claim text: {raw_train.claims[raw_train.article_claim[instance_no]]}")
print(f"Article text: {raw_train.articles[instance_no]}")
print(f"Stance: {raw_train.stances[instance_no]}")

Claim text: بمناسبة العام الجديد  شركة ليكزس توزع 200 سيارة مجانا
Article text: كثيرا ما تداولت صحف ومواقع إخبارية تقارير عن الهدايا التي منحتها الملكة إليزابيث الثانية (  ) ملكة بريطانيا، للعاملين لديها بمناسبة أعياد الميلاد، إلا أن صور هذه الهدايا وطبيعتها لم تكشف بشكل كامل إلا مؤخرا، وتحديدا بعد أن كشف عنها أحد جامعي التذكارات الملكية، ويدعى إيان شابيرو ( )، ويمتلك إيان مجموعة من الهدايا الملكية التي قدمتها الملكة للعاملين لديها، وتتضمن وعاء أنيق من الكريستال وطاقم عبوات الملح والفلفل الخاصة بالمائدة وإطار صور أنيق يحمل صورة رسمية للملكة التقطت بمناسبة عيد ميلادها الثمانين. مجموعة هدايا الملكة للعاملين لديها منذ عام 2002 طبقا لما نشره موقع   فإن مجموعة الهدايا الملكية بمناسبة أعياد الميلاد والتي قام باقتنائها أيان شابيرو، تتضمن مجموعة من الهدايا الشخصية التي قدمتها الملكة لعدد من العاملين لديها في قصر باكنغهام وقلعة وندسور بمناسبة أعياد الميلاد خلال الفترة ما بين عامي 2002-2015، إلى جانب عدد من الهدايا التي اعتادت ملكة بريطانيا تقديمها لجميع العاملين لديها في كل عام مثل بودنج أعياد 

- Thus, the instances are triplets, Claim/Article/Stance.
- Note that the original language of the data is Arabic.

## Dataset

In [ ]:
batch_size = 64
sequence_length = 512
checkpoint = 'bert-base-multilingual-cased'

- Download the tokenizer:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize(instance):
  return tokenizer(instance[0], instance[1], truncation=True, padding='max_length', max_length=sequence_length)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

- Train dataloader:

In [ ]:
train_labels = [stance_to_int[stance] for stance in raw_train.stances]
train_claims = list(map(raw_train.claims.__getitem__, raw_train.article_claim))
train_encodings = tokenize((train_claims, raw_train.articles))
train_dataset = CustomDataset(train_encodings, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

- Val dataloader

In [ ]:
val_labels = [stance_to_int[stance] for stance in raw_val.stances]
val_claims = list(map(raw_val.claims.__getitem__, raw_val.article_claim))
val_encodings = tokenize((val_claims, raw_val.articles))
val_dataset = CustomDataset(val_encodings, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

- Test dataloader

In [ ]:
test_labels = [stance_to_int[stance] for stance in raw_test.stances]
test_claims = list(map(raw_test.claims.__getitem__, raw_test.article_claim))
test_encodings = tokenize((test_claims, raw_test.articles))
test_dataset = CustomDataset(test_encodings, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

- Check a batch from the data:

In [ ]:
for batch in test_dataloader:
  break
print({k: v.shape for k,v in batch.items()})

{'input_ids': torch.Size([64, 512]), 'token_type_ids': torch.Size([64, 512]), 'attention_mask': torch.Size([64, 512]), 'labels': torch.Size([64])}


## Model

- Download the model:

In [ ]:
pretrained_model = AutoModel.from_pretrained(checkpoint, output_hidden_states=True, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
class Model(torch.nn.Module):
  def __init__(self, pretrained_model):
    super(Model, self).__init__()
    self.pretrained_model = pretrained_model
    for param in self.pretrained_model.parameters():
      param.requires_grad = False
    self.conv1 = torch.nn.Conv1d(in_channels=768, out_channels=100, kernel_size=2)
    self.conv2 = torch.nn.Conv1d(in_channels=768, out_channels=100, kernel_size=3)
    self.conv3 = torch.nn.Conv1d(in_channels=768, out_channels=100, kernel_size=4)
    self.pool1 = torch.nn.MaxPool1d(kernel_size=sequence_length*4-1)
    self.pool2 = torch.nn.MaxPool1d(kernel_size=sequence_length*4-2)
    self.pool3 = torch.nn.MaxPool1d(kernel_size=sequence_length*4-3)
    self.dropout = torch.nn.Dropout(p=0.3)
    self.classifier = torch.nn.Linear(in_features=300, out_features=4)

  def forward(self, inputs):
    encoder_outputs = self.pretrained_model(**inputs)['hidden_states']
    l12_outputs = encoder_outputs[12]
    l11_outputs = encoder_outputs[11]
    l10_outputs = encoder_outputs[10]
    l9_outputs = encoder_outputs[9]
    encoder_outputs = torch.cat((l9_outputs, l10_outputs, l11_outputs, l12_outputs), dim=1)
    encoder_outputs = encoder_outputs.permute(0, 2, 1)
    conv1_outputs = self.conv1(encoder_outputs)
    conv2_outputs = self.conv2(encoder_outputs)
    conv3_outputs = self.conv3(encoder_outputs)
    pool1_outputs = self.pool1(conv1_outputs)
    pool2_outputs = self.pool2(conv2_outputs)
    pool3_outputs = self.pool3(conv3_outputs)
    concat_outputs = torch.cat((pool1_outputs, pool2_outputs, pool3_outputs), dim=1).squeeze()
    concat_outputs = self.dropout(concat_outputs)
    logits = self.classifier(concat_outputs)
    return logits

- Create a model instance and put it on the GPU:

In [ ]:
model = Model(pretrained_model)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

cuda


- Print Model trainable parameters:

In [ ]:
for param in model.parameters():
  if param.requires_grad:
    print(param.size())

torch.Size([100, 768, 2])
torch.Size([100])
torch.Size([100, 768, 3])
torch.Size([100])
torch.Size([100, 768, 4])
torch.Size([100])
torch.Size([4, 300])
torch.Size([4])


## Training

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
  running_loss, running_corrects = 0, 0

  progress_bar = tqdm(range(len(dataloader)))
  model.train()
  for batch in dataloader:
    input_ids, token_type_ids, attention_mask, labels = batch.values()
    labels = labels.to(device)
    inputs = {'input_ids': input_ids.to(device),
              'token_type_ids': token_type_ids.to(device),
              'attention_mask': attention_mask.to(device)}
    logits = model(inputs)
    loss = loss_fn(logits, labels)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    progress_bar.update(1)

    predictions = torch.argmax(logits, dim=-1)

    running_loss += loss.item() * len(labels)
    running_corrects += torch.sum(predictions == labels)

  final_loss = running_loss / len(dataloader.dataset)
  accuracy = running_corrects / len(dataloader.dataset)
  return final_loss, accuracy

In [ ]:
def test_loop(dataloader, model, loss_fn):
  running_loss, running_corrects = 0, 0
  all_preds = []

  model.eval()
  for batch in dataloader:
    input_ids, token_type_ids, attention_mask, labels = batch.values()
    labels = labels.to(device)
    inputs = {'input_ids': input_ids.to(device),
              'token_type_ids': token_type_ids.to(device),
              'attention_mask': attention_mask.to(device)}
    with torch.no_grad():
      logits = model(inputs)
      loss = loss_fn(logits, labels)

    predictions = torch.argmax(logits, dim=-1)

    running_loss += loss.item() * len(labels)
    running_corrects += torch.sum(predictions == labels)

    all_preds = np.concatenate((all_preds, predictions.cpu().numpy()))

  final_loss = running_loss / len(dataloader.dataset)
  accuracy = running_corrects / len(dataloader.dataset)
  f1score = f1_score(dataloader.dataset.labels, all_preds, average=None)
  mf1score = f1_score(dataloader.dataset.labels, all_preds, average='macro')

  return final_loss, accuracy, f1score, mf1score

In [ ]:
class EarlyStopping:
  def __init__(self, patience=1, min_delta=0):
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation_loss = float('inf')

  def __call__(self, validation_loss):
    if validation_loss < self.min_validation_loss:
      self.min_validation_loss = validation_loss
      self.counter = 0
    elif validation_loss > (self.min_validation_loss + self.min_delta):
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False

In [ ]:
epochs = 100
lr = 3e-4
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=lr)
early_stopping = EarlyStopping(patience=3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss, train_accuracy = train_loop(train_dataloader, model, loss_fn, optimizer)
    val_loss, val_accuracy, _, _ = test_loop(val_dataloader, model, loss_fn)

    print(f"Train_loss: {train_loss:.3f}, Train_acc: {train_accuracy:.3f}",
          f"Val_loss: {val_loss:.3f}, Val_accuracy: {val_accuracy:.3f}",
          "\n-------------------------------")

    if early_stopping(val_loss):
      print("Early stopping!")
      break

Epoch 1
-------------------------------


  0%|          | 0/45 [00:00<?, ?it/s]

Train_loss: 1.178, Train_acc: 0.532 Val_loss: 1.082, Val_accuracy: 0.599 
-------------------------------
Epoch 2
-------------------------------


  0%|          | 0/45 [00:00<?, ?it/s]

Train_loss: 0.839, Train_acc: 0.691 Val_loss: 0.834, Val_accuracy: 0.696 
-------------------------------
Epoch 3
-------------------------------


...

Train_loss: 0.122, Train_acc: 0.972 Val_loss: 0.589, Val_accuracy: 0.828 
-------------------------------
Epoch 16
-------------------------------


  0%|          | 0/45 [00:00<?, ?it/s]

Train_loss: 0.114, Train_acc: 0.971 Val_loss: 0.569, Val_accuracy: 0.830 
-------------------------------
Epoch 17
-------------------------------


  0%|          | 0/45 [00:00<?, ?it/s]

Train_loss: 0.100, Train_acc: 0.975 Val_loss: 0.600, Val_accuracy: 0.831 
-------------------------------
Early stopping!


## Evaluation

In [ ]:
_, val_accuracy, val_f1score, val_mf1score = test_loop(val_dataloader, model, loss_fn)

print("Validation Resutls:")
print("=====================")
print(f"Accuracy: {val_accuracy:.3f}")
agree, disagree, discuss, unrelated = val_f1score
print("Per Class F1 scores:")
print(f"Agree   : {agree:.3f}")
print(f"Disagree: {disagree:.3f}")
print(f"Discuss : {discuss:.3f}")
print(f"Unrelated: {unrelated:.3f}")
print(f"Macro F1 scores: {val_mf1score:.3f}")

Validation Resutls:
Accuracy: 0.831
Per Class F1 scores:
Agree   : 0.864
Disagree: 0.735
Discuss : 0.505
Unrelated: 0.896
Macro F1 scores: 0.750


In [ ]:
_, test_accuracy, test_f1score, test_mf1score = test_loop(test_dataloader, model, loss_fn)

print("Testing Resutls:")
print("=====================")
print(f"Accuracy: {test_accuracy:.3f}")
agree, disagree, discuss, unrelated = test_f1score
print("Per Class F1 scores:")
print(f"Agree   : {agree:.3f}")
print(f"Disagree: {disagree:.3f}")
print(f"Discuss : {discuss:.3f}")
print(f"Unrelated: {unrelated:.3f}")
print(f"Macro F1 scores: {test_mf1score:.3f}")

Testing Resutls:
Accuracy: 0.861
Per Class F1 scores:
Agree   : 0.850
Disagree: 0.780
Discuss : 0.451
Unrelated: 0.934
Macro F1 scores: 0.754
